In [ ]:
import networkx as nx
import pandas as pd


import pydot
from networkx.drawing.nx_pydot import graphviz_layout

from community import community_louvain as community

In [ ]:
#base_df = pd.read_csv("/data/schwurbelarchiv/extracted_information/dataframes/base_df.csv.gzip", compression = 'gzip')

In [ ]:
a_nodes = base_df.query("message_hash in fwd_message_hash and not (message_hash != message_hash)") 

In [ ]:
#defining sample size, as full dataset takes longer than 12 hours
a_nodes = a_nodes.head(2000)

In [ ]:
a_tup = tuple([a_nodes['group_name'].tolist(), a_nodes['message_hash'].tolist(), a_nodes['author'].tolist()])

In [ ]:
a_edge = []
for i in range(len(a_tup[0])):
    try:
        tmp = base_df['group_name'][(base_df['fwd_message_hash'] == a_tup[1][i]) & (base_df['fwd_author'] == a_tup[2][i])]
        a = tmp.tolist()[0]
        
    except:
        continue
    
    a_edge.append((a_tup[0][i], a))

In [ ]:
G = nx.complete_graph(1)

G.add_nodes_from(a_tup[0])

G.add_edges_from(a_edge)

nodes_to_remove = [node for node in G.nodes() if G.degree(node) < 1]

G.remove_nodes_from(nodes_to_remove)

In [ ]:
partition = community.best_partition(G)

In [ ]:
pos = nx.spring_layout(G)
fig, ax = plt.subplots(figsize=(24,20))

nx.draw(G, pos, ax = ax, with_labels=False, node_size=30, cmap=plt.cm.jet, node_color=list(partition.values()))
plt.show()

In [ ]:
# Compute the best partition
partition = community.best_partition(G)

# Draw the graph
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=30,
                       cmap=plt.cm.jet, node_color=list(partition.values()))
nx.draw_networkx_edges(G, pos, alpha=0.7)



In [ ]:
#mean degree
degrees = [val for (node, val) in G.degree()]
sum(degrees)/len(degrees)

In [ ]:
#modularity
mod = community.modularity(partition, G)

In [ ]:
#centrality
deg_centrality = nx.degree_centrality(G)

cent = []
for i in deg_centrality:
    cent.append(deg_centrality[i])

In [ ]:
sum(cent)/len(cent)